In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "2" 
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments
from transformers import pipeline
from datasets import load_dataset
from huggingface_hub import hf_hub_download
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to C:\Users\researcher\.huggingface\token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


# Dataset

In [3]:
data_files = {"steamclean_processed_300t.csv"}
training_dataset=load_dataset("HFFErica/labelled", data_files=data_files, split="train[:240000]")
validation_dataset=load_dataset("HFFErica/labelled", data_files=data_files, split="train[240000:300000]")

print (training_dataset)
print(validation_dataset)

Using custom data configuration HFFErica--labelled-5d4dba2d06e4f6ce
Found cached dataset csv (C:/Users/researcher/.cache/huggingface/datasets/HFFErica___csv/HFFErica--labelled-5d4dba2d06e4f6ce/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
Using custom data configuration HFFErica--labelled-5d4dba2d06e4f6ce
Found cached dataset csv (C:/Users/researcher/.cache/huggingface/datasets/HFFErica___csv/HFFErica--labelled-5d4dba2d06e4f6ce/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


Dataset({
    features: ['game_id', 'content', 'label_1', 'label_2', 'preprocessed', 'Column1'],
    num_rows: 240000
})
Dataset({
    features: ['game_id', 'content', 'label_1', 'label_2', 'preprocessed', 'Column1'],
    num_rows: 60000
})


In [4]:
data_files_test = {'test':"testdataset_binarylabel_results2_5000.csv"}
test_dataset_one=load_dataset("HFFErica/labelled", data_files=data_files_test, split="test[:1000]")

Using custom data configuration HFFErica--labelled-72b0efcff31242ba
Found cached dataset csv (C:/Users/researcher/.cache/huggingface/datasets/HFFErica___csv/HFFErica--labelled-72b0efcff31242ba/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


In [5]:
from datasets import Dataset, Value, ClassLabel, Features

features = training_dataset.features.copy()
features["label_1"] = ClassLabel(names=["negative", "positive"])

def adjust_labels(batch):
    batch["label_1"] = [sentiment for sentiment in batch["label_1"]]
    return batch
training_dataset = training_dataset.map(adjust_labels, batched=True, features=features)

Loading cached processed dataset at C:/Users/researcher/.cache/huggingface/datasets/HFFErica___csv/HFFErica--labelled-5d4dba2d06e4f6ce/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a\cache-0375bafbb7a7a16a.arrow


In [6]:
features = validation_dataset.features.copy()
features["label_1"] = ClassLabel(names=["negative", "positive"])

def adjust_labels(batch):
    batch["label_1"] = [sentiment for sentiment in batch["label_1"]]
    return batch
validation_dataset = validation_dataset.map(adjust_labels, batched=True, features=features)

  0%|          | 0/60 [00:00<?, ?ba/s]

In [7]:
features = test_dataset_one.features.copy()
features["binarylabel_RoBERTa-large.1"] = ClassLabel(names=["negative", "positive"])
features["DistilBERT-base-uncased.1"] = ClassLabel(names=["negative", "positive"])
features["recommend_labels"] = ClassLabel(names=["notrecommend", "recommend"])


def adjust_labels(batch):
    batch["binarylabel_RoBERTa-large.1"] = [sentiment for sentiment in batch["binarylabel_RoBERTa-large.1"]]
    batch["DistilBERT-base-uncased.1"] = [sentiment for sentiment in batch["DistilBERT-base-uncased.1"]]
    batch["recommend_labels"] = [sentiment for sentiment in batch["recommend_labels"]]

    return batch
test_dataset_one = test_dataset_one.map(adjust_labels, batched=True, features=features)

  0%|          | 0/1 [00:00<?, ?ba/s]

# Model & tokenizer

In [8]:
training_dataset = training_dataset.map(lambda examples: {'labels': examples['label_1']}, batched=True)
validation_dataset = validation_dataset.map(lambda examples: {'labels': examples['label_1']}, batched=True)
test_dataset_one = test_dataset_one.map(lambda examples: {'labels': examples['DistilBERT-base-uncased.1']}, batched=True)

Loading cached processed dataset at C:/Users/researcher/.cache/huggingface/datasets/HFFErica___csv/HFFErica--labelled-5d4dba2d06e4f6ce/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a\cache-9b7698b214db3105.arrow


  0%|          | 0/60 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [9]:
# model_id = 'prajjwal1/bert-tiny'
model_id = 'distilbert-base-uncased-finetuned-sst-2-english'


model = AutoModelForSequenceClassification.from_pretrained(model_id, 
            num_labels=training_dataset.features["label_1"].num_classes)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [10]:
MAX_LENGTH = 100
training_dataset = training_dataset.map(lambda e: tokenizer(e['preprocessed'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)
validation_dataset = validation_dataset.map(lambda e: tokenizer(e['preprocessed'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)
test_dataset_one = test_dataset_one.map(lambda e: tokenizer(e['replace_name'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)

Loading cached processed dataset at C:/Users/researcher/.cache/huggingface/datasets/HFFErica___csv/HFFErica--labelled-5d4dba2d06e4f6ce/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a\cache-aee50aea6ffb6b53.arrow


  0%|          | 0/60 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
training_dataset.set_format(type='torch', columns=['input_ids',  'attention_mask', 'labels'])
validation_dataset.set_format(type='torch', columns=['input_ids',  'attention_mask', 'labels'])
test_dataset_one.set_format(type='torch', columns=['input_ids',  'attention_mask', 'labels'])

# Training

In [12]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [13]:
training_args = TrainingArguments(
    output_dir='./sentimentbinaryresults',          # output directory
    learning_rate=3e-4,
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    do_train=True,
    do_eval=True,
    no_cuda=False,
    load_best_model_at_end=True,
    eval_steps=100,
    evaluation_strategy="steps"
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=training_dataset,         # training dataset
    eval_dataset=validation_dataset,            # evaluation dataset
    compute_metrics=compute_metrics
)

train_out = trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Column1, game_id, label_1, content, label_2, preprocessed. If Column1, game_id, label_1, content, label_2, preprocessed are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 240000
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 7500
Automatic Weights & Biases logging enabled, to disa

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.442400,0.344365,0.852217,0.851996,0.853978,0.852078
200,0.341500,0.310443,0.870817,0.870646,0.872436,0.870687
300,0.383000,0.338970,0.871850,0.871725,0.873626,0.871986
400,0.443900,0.426507,0.851417,0.851251,0.852686,0.851299
500,0.432600,0.491526,0.808500,0.804269,0.836233,0.807934
600,0.565700,0.529112,0.772317,0.763544,0.817767,0.771571
700,0.566900,0.588136,0.745017,0.741165,0.759496,0.744548
800,0.643200,0.614998,0.672100,0.654832,0.717223,0.672990
900,0.615200,0.568364,0.718550,0.705808,0.766408,0.719380
1000,0.565700,0.599179,0.682267,0.651225,0.786960,0.683451


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Column1, game_id, label_1, content, label_2, preprocessed. If Column1, game_id, label_1, content, label_2, preprocessed are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 60000
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Column1, game_id, label_1, content, label_2, preprocessed. If Column1, game_id, label_1, content, label_2, preprocessed are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 60000
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenc

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Column1, game_id, label_1, content, label_2, preprocessed. If Column1, game_id, label_1, content, label_2, preprocessed are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 60000
  Batch size = 32
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Column1, game_id, label_1, content, label_2, preprocessed. If Column1, game

***** Running Evaluation *****
  Num examples = 60000
  Batch size = 32
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Column1, game_id, label_1, content, label_2, preprocessed. If Column1, game_id, label_1, content, label_2, preprocessed are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 60000
  Batch size = 32
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined

  Num examples = 60000
  Batch size = 32
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Column1, game_id, label_1, content, label_2, preprocessed. If Column1, game_id, label_1, content, label_2, preprocessed are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 60000
  Batch size = 32
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels

  Batch size = 32
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Column1, game_id, label_1, content, label_2, preprocessed. If Column1, game_id, label_1, content, label_2, preprocessed are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 60000
  Batch size = 32
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samp

D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Column1, game_id, label_1, content, label_2, preprocessed. If Column1, game_id, label_1, content, label_2, preprocessed are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 60000
  Batch size = 32
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_div

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Column1, game_id, label_1, content, label_2, preprocessed. If Column1, game_id, label_1, content, label_2, preprocessed are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 60000
  Batch size = 32
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Column1, game_id, label_1, content, label_2, preprocessed. If Column1, game

In [14]:
trainer.save_model("./models/sa_model")

Saving model checkpoint to ./models/sa_model
Configuration saved in ./models/sa_model\config.json
Model weights saved in ./models/sa_model\pytorch_model.bin


# Pipeline

In [15]:
# from transformers import BertConfig, BertModel
# # if model is on hugging face Hub
# model = BertModel.from_pretrained("bert-base-uncased")
# # from local folder
# model = BertModel.from_pretrained("./test/saved_model/")

In [18]:
model = AutoModelForSequenceClassification.from_pretrained("./models/sa_model/")
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

loading configuration file ./models/sa_model/config.json
Model config DistilBertConfig {
  "_name_or_path": "./models/sa_model/",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "vocab_size": 30522
}

loading weights file ./models/sa_model/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSe

In [44]:
test_examples = pd.read_csv('D:\Projects\Jupyter\Github Docs\datasets\preprocessed_word_correct_token.csv', encoding='utf-8')
test_examples.head()

,id,name,content,sentence,review_score,review_votes,readability,preprocessed_reviews,wordtoken,correctmapping,ngrams_reviews,ngrams_token,lda_reviews,lda_token
0,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,This was my review prior to the 1.1 update ___...,1,1,17.6,this be my review prior to the update nothis g...,"['this', 'be', 'my', 'review', 'prior', 'to', ...","['this', 'be', 'my', 'review', 'prior', 'to', ...",this be my review prior to the update nothin g...,"['review', 'prior', 'update', 'nothin', 'fail'...",review prior update nothin fail miserably anno...,"['review', 'prior', 'update', 'nothin', 'fail'..."
1,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,"When the game first came out it had over 200,0...",1,1,6.5,when the game first come out it have over play...,"['when', 'the', 'game', 'first', 'come', 'out'...","['when', 'the', 'game', 'first', 'come', 'out'...",when the game first come out it have over play...,"['first', 'player', 'one', 'point', 'mear']",first come player one point mear,"['first', 'player', 'one', 'point', 'mear']"
2,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,If that doesn't prove this game is the biggest...,1,1,5.6,if that do not prove this game be the big scam...,"['if', 'that', 'do', 'not', 'prove', 'this', '...","['if', 'that', 'do', 'not', 'prove', 'this', '...",if that do not prove this game be the big scam...,"['prove', 'this_game', 'big', 'scam', 'know']",prove big scam know,"['prove', 'big', 'scam', 'know']"
3,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,The devs completely lied about countless featu...,1,1,8.7,the developer completely lie about countless f...,"['the', 'developer', 'completely', 'lie', 'abo...","['the', 'developer', 'completely', 'lie', 'abo...",the developer completely lie about countless f...,"['the_developer', 'completely', 'lie_about', '...",developer completely lie countless feature meant,"['developer', 'completely', 'lie', 'countless'..."
4,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,______________________________________________...,1,1,29.1,i clearly have very strong feel about the game...,"['clearly', 'have', 'very', 'strong', 'feel', ...","['clearly', 'have', 'very', 'strong', 'feel', ...",clearly have very strong feel about the game a...,"['clearly', 'strong', 'feel', 'play', 'hour', ...",clearly strong feel play hour hello games rele...,"['clearly', 'strong', 'feel', 'play_hour', 'he..."


In [49]:
df1=test_examples['preprocessed_reviews'][:10]
df1

0    this be my review prior to the update nothis g...
1    when the game first come out it have over play...
2    if that do not prove this game be the big scam...
3    the developer completely lie about countless f...
4    i clearly have very strong feel about the game...
5    obviously the update do not add all the featur...
6    it show that hello games s have start take a s...
7    i really do enjoy the game when it first be re...
8    it be very nice to see the developer go back t...
9    thought about the update game mode the additon...
Name: preprocessed_reviews, dtype: object

In [50]:
results = classifier(df1.values.tolist())

Disabling tokenizer parallelism, we're using DataLoader multithreading already


In [51]:
results

[{'label': 'NEGATIVE', 'score': 0.920569121837616},
 {'label': 'POSITIVE', 'score': 0.8704404830932617},
 {'label': 'NEGATIVE', 'score': 0.6774666905403137},
 {'label': 'NEGATIVE', 'score': 0.9185805320739746},
 {'label': 'NEGATIVE', 'score': 0.8531467318534851},
 {'label': 'NEGATIVE', 'score': 0.8237114548683167},
 {'label': 'NEGATIVE', 'score': 0.8814191818237305},
 {'label': 'NEGATIVE', 'score': 0.6638973951339722},
 {'label': 'NEGATIVE', 'score': 0.9155842065811157},
 {'label': 'POSITIVE', 'score': 0.8704395890235901}]